In [ ]:
from PIL import Image
import requests
import os
import json
import matplotlib.pyplot as plt

from lib import get_ids, get_images, get_titles

from transformers import CLIPProcessor, CLIPModel

from torch import nn
import torch
import torch.functional as F

In [ ]:
search_file = "searches/full_100_2022-03-21_17:18:01.json"
image_dir = "images/"

In [ ]:
clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_embeds(text, images):
    inputs = processor(text=text, images=images, return_tensors="pt", padding=True)
    outputs = clip(**inputs)
    text_embeds = outputs.text_embeds
    image_embeds = outputs.image_embeds
    # concat
    embeds = torch.cat((text_embeds, image_embeds), dim=1)
    return embeds

In [ ]:
class PriceModel(nn.Module):

    def __init__(self):
        super(PriceModel).__init__()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)

    def forward(self, embeds):            
        # pass through layers
        z = F.relu(self.fc1(embeds))
        z = F.relu(self.fc2(z))
        z = self.fc3(z)
        return z

In [ ]:
# read search_file
with open(search_file, "r") as f:
    search = json.load(f)

ids = get_ids(search)
titles = get_titles(search)
images = get_images(search)